In [ ]:
# DONT RUNT THIS CELL
# moved to the beginning to not lost it in case we need it later


#drop columns i dont care or have too many empty values:
aux_training = aux_training.drop('date_time', axis=1)

aux_training = aux_training.drop('comp1_rate', axis=1)
aux_training = aux_training.drop('comp1_inv', axis=1)
aux_training = aux_training.drop('comp1_rate_percent_diff', axis=1)

aux_training = aux_training.drop('comp2_rate', axis=1)
aux_training = aux_training.drop('comp2_inv', axis=1)
aux_training = aux_training.drop('comp2_rate_percent_diff', axis=1)

aux_training = aux_training.drop('comp3_rate', axis=1)
aux_training = aux_training.drop('comp3_inv', axis=1)
aux_training = aux_training.drop('comp3_rate_percent_diff', axis=1)

aux_training = aux_training.drop('comp4_rate', axis=1)
aux_training = aux_training.drop('comp4_inv', axis=1)
aux_training = aux_training.drop('comp4_rate_percent_diff', axis=1)

aux_training = aux_training.drop('comp5_rate', axis=1)
aux_training = aux_training.drop('comp5_inv', axis=1)
aux_training = aux_training.drop('comp5_rate_percent_diff', axis=1)

aux_training = aux_training.drop('comp6_rate', axis=1)
aux_training = aux_training.drop('comp6_inv', axis=1)
aux_training = aux_training.drop('comp6_rate_percent_diff', axis=1)

aux_training = aux_training.drop('comp7_rate', axis=1)
aux_training = aux_training.drop('comp7_inv', axis=1)
aux_training = aux_training.drop('comp7_rate_percent_diff', axis=1)

aux_training = aux_training.drop('comp8_rate', axis=1)
aux_training = aux_training.drop('comp8_inv', axis=1)
aux_training = aux_training.drop('comp8_rate_percent_diff', axis=1)

aux_training = aux_training.drop('visitor_hist_starrating', axis=1)
aux_training = aux_training.drop('visitor_hist_adr_usd', axis=1)

aux_training = aux_training.drop('srch_query_affinity_score', axis=1)

aux_training = aux_training.drop('gross_bookings_usd', axis=1)

aux_training = aux_training.drop('prop_review_score', axis=1)
aux_training = aux_training.drop('prop_location_score2', axis=1)
aux_training = aux_training.drop('orig_destination_distance', axis=1)

In [ ]:
pip install lightgbm

In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import copy
from tabulate import tabulate
from pathlib import Path
from cProfile import label
from jinja2 import Undefined
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn import metrics,tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from datamining_machine import DataMiningMachine
from sklearn.model_selection import train_test_split

In [2]:
test_data  = pd.read_csv("test_set_VU_DM.csv")

In [3]:
training_data  = pd.read_csv("training_set_VU_DM.csv")

In [ ]:
training_data.head()

In [4]:
#create new column for NDCG evaluation
training_data['target_col'] = training_data.apply(
    lambda row: (5 if row.booking_bool == 1 else (1 if row.click_bool == 1 else 0)), axis=1)

In [5]:
aux_training = copy.deepcopy(training_data)

In [ ]:
aux_training.head()

In [ ]:
print("len target 0: ",len(aux_training.loc[aux_training["target_col"] == 0]))
print("len target 1: ",len(aux_training.loc[aux_training["target_col"] == 1]))
print("len target 5: ",len(aux_training.loc[aux_training["target_col"] == 5]))


In [ ]:
## get set balanced for target column
#np.random.seed(0)
#balanced = aux_training.groupby('target_col')
#balanced = balanced.apply(lambda x: x.sample(balanced.size().min()))

print("len target 0: ",len(balanced.loc[balanced["target_col"] == 0]))
print("len target 1: ",len(balanced.loc[balanced["target_col"] == 1]))
print("len target 5: ",len(balanced.loc[balanced["target_col"] == 5]))

In [ ]:
#np.random.seed(0)
#rand_sample2 = training_data.groupby('booking_bool').sample(frac=.5) #random
#rand_sample2.head(50)

In [ ]:
# np.random.seed(0)
#rand_sample = training_data.groupby('srch_id').sample(n=5) #random sample keeping 5 rows for any srch_id

## SAMPLE KEEPING SRCH_IDs OR BOOKING BOOL?

In [6]:
aux_training = aux_training.sample(frac=.5) #random sample

print("len target 0: ",len(aux_training.loc[aux_training["target_col"] == 0]))
print("len target 1: ",len(aux_training.loc[aux_training["target_col"] == 1]))
print("len target 5: ",len(aux_training.loc[aux_training["target_col"] == 5]))

len target 0:  2368330
len target 1:  41474
len target 5:  69370


In [7]:
aux_training = aux_training.drop('date_time', axis=1) # cause me dont datee

In [8]:
##  drop columns that are not in test set (??? ) 
aux_training = aux_training.drop('click_bool', axis=1)   #replaced by target_col and not in test set 
aux_training = aux_training.drop('booking_bool', axis=1) #replaced by target_col and not in test set 


In [ ]:
# CAN WE DROP THIS AFTER TRAINING?
#aux_training = aux_training.drop('position', axis=1)          # (not in test set)not sure about drop this one. Maybe some transformation ??
#aux_training = aux_training.drop('gross_booking_usd', axis=1) # (not in test set)not sure about drop this one. Maybe some transformation ??

In [ ]:
print("Empty values per column: \n",aux_training.isnull().sum())

In [ ]:
aux_training.head()

In [9]:
feature = aux_training.drop('target_col', axis=1)
target  =  aux_training['target_col']

In [ ]:
feature.head()

In [10]:
X_train, X_val, y_train, y_val = train_test_split(feature, target, test_size=0.2, random_state=0)

print("X_train.shape = ",X_train.shape)
print("X_val.shape = ",X_val.shape)
print("\ny_train.shape = ",y_train.shape)
print("y_val.shape = ",y_val.shape)

    

X_train.shape =  (1983339, 51)
X_val.shape =  (495835, 51)

y_train.shape =  (1983339,)
y_val.shape =  (495835,)


In [ ]:
X_train.head()

In [11]:
X_train = X_train.sort_index()
y_train = y_train.sort_index()

X_val = X_val.sort_index()
y_val = y_val.sort_index()

In [12]:
qids_train = X_train.groupby("srch_id")["srch_id"].count().to_numpy()
qids_val   = X_val.groupby("srch_id")["srch_id"].count().to_numpy()


In [ ]:
qids_train

In [ ]:
print("query train ids mean: ",qids_train.mean()) # should we ensure there are enough rows from same srch_id ??? ANS: apparently not
print("query val   ids mean: ",qids_val.mean()) 

In [ ]:
#X_train = X_train.drop('srch_id', axis=1) #do we need to drop search id?
#X_val   = X_val.drop('srch_id', axis=1)   #do we need to drop search id?

In [13]:
FIXED_PARAMS={
              'objective'             : 'lambdarank',
              #'objective'               : 'rank_xendcg',
              'metric'                : 'ndcg',
              'boosting_type'         : 'gbdt',
              'is_unbalance'          : True,
#               'is_unbalance'          : False,
              'num_iterations'        : 300,
              'early_stopping_rounds' : 30}

SEARCH_PARAMS = {'learning_rate': 0.3,
                 'max_depth': 15,
                 'num_leaves': 20,
                 'feature_fraction': 0.8,
                 'subsample': 0.2}

gbm = lgb.LGBMRanker( 
        objective               = FIXED_PARAMS["objective"],
        metric                  = FIXED_PARAMS["metric"],
        boosting_type           = FIXED_PARAMS["boosting_type"],
        is_unbalance            = FIXED_PARAMS["is_unbalance"],
        num_iterations          = FIXED_PARAMS["num_iterations"],
        early_stopping_rounds   = FIXED_PARAMS["early_stopping_rounds"],
        learning_rate           = SEARCH_PARAMS["learning_rate"],
        max_depth               = SEARCH_PARAMS["max_depth"],
        num_leaves              = SEARCH_PARAMS["num_leaves"],
        feature_fraction        = SEARCH_PARAMS["feature_fraction"],
        subsample               = SEARCH_PARAMS["subsample"],
)

# gbm = lgb.LGBMRanker( params )




In [14]:
gbm.fit(X_train, 
        y_train, 
        group=qids_train,
        eval_set=[(X_val, y_val)], 
        eval_group=[qids_val],
        eval_at=[5]
)

C:\Users\Martin\anaconda3\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] early_stopping_round is set=30, early_stopping_rounds=30 will be ignored. Current value: early_stopping_round=30
[1]	valid_0's ndcg@5: 0.9903
[2]	valid_0's ndcg@5: 0.990935
[3]	valid_0's ndcg@5: 0.99115
[4]	valid_0's ndcg@5: 0.991289
[5]	valid_0's ndcg@5: 0.991359
[6]	valid_0's ndcg@5: 0.991461
[7]	valid_0's ndcg@5: 0.99155
[8]	valid_0's ndcg@5: 0.991636
[9]	valid_0's ndcg@5: 0.99166
[10]	valid_0's ndcg@5: 0.99172
[11]	valid_0's ndcg@5: 0.991763
[12]	valid_0's ndcg@5: 0.991744
[13]	valid_0's ndcg@5: 0.991737
[14]	valid_0's ndcg@5: 0.991764
[15]	valid_0's ndcg@5: 0.991723
[16]	valid_0's ndcg@5: 0.991752
[17]	valid_0's ndcg@5: 0.991783
[18]	valid_0's ndcg@5: 0.991815
[19]	valid_0's ndcg@5: 0.991794
[20]	valid_0's ndcg@5: 0.991769
[21]	valid_0's ndcg@5: 0.991794
[22]	valid_0's ndcg@5: 0.991808
[23]	valid_0's ndcg@5: 0.991834
[24]	valid_0's ndcg@5

LGBMRanker(early_stopping_rounds=30, feature_fraction=0.8, is_unbalance=True,
           learning_rate=0.3, max_depth=15, metric='ndcg', num_iterations=300,
           num_leaves=20, objective='lambdarank', subsample=0.2)

In [ ]:
#test_pred = gbm.predict(X_test)

#print(test_pred)

In [ ]:
X_train.columns

In [ ]:
aux_test = test_data

In [ ]:
#drop columns i dont care or have too many empty values:


aux_test = aux_test.drop('date_time', axis=1)

aux_test = aux_test.drop('comp1_rate', axis=1)
aux_test = aux_test.drop('comp1_inv', axis=1)
aux_test = aux_test.drop('comp1_rate_percent_diff', axis=1)

aux_test = aux_test.drop('comp2_rate', axis=1)
aux_test = aux_test.drop('comp2_inv', axis=1)
aux_test = aux_test.drop('comp2_rate_percent_diff', axis=1)

aux_test = aux_test.drop('comp3_rate', axis=1)
aux_test = aux_test.drop('comp3_inv', axis=1)
aux_test = aux_test.drop('comp3_rate_percent_diff', axis=1)

aux_test = aux_test.drop('comp4_rate', axis=1)
aux_test = aux_test.drop('comp4_inv', axis=1)
aux_test = aux_test.drop('comp4_rate_percent_diff', axis=1)

aux_test = aux_test.drop('comp5_rate', axis=1)
aux_test = aux_test.drop('comp5_inv', axis=1)
aux_test = aux_test.drop('comp5_rate_percent_diff', axis=1)

aux_test = aux_test.drop('comp6_rate', axis=1)
aux_test = aux_test.drop('comp6_inv', axis=1)
aux_test = aux_test.drop('comp6_rate_percent_diff', axis=1)

aux_test = aux_test.drop('comp7_rate', axis=1)
aux_test = aux_test.drop('comp7_inv', axis=1)
aux_test = aux_test.drop('comp7_rate_percent_diff', axis=1)

aux_test = aux_test.drop('comp8_rate', axis=1)
aux_test = aux_test.drop('comp8_inv', axis=1)
aux_test = aux_test.drop('comp8_rate_percent_diff', axis=1)

aux_test = aux_test.drop('visitor_hist_starrating', axis=1)
aux_test = aux_test.drop('visitor_hist_adr_usd', axis=1)

aux_test = aux_test.drop('srch_query_affinity_score', axis=1)

#aux_test = aux_test.drop('gross_bookings_usd', axis=1)

aux_test = aux_test.drop('prop_review_score', axis=1)
aux_test = aux_test.drop('prop_location_score2', axis=1)
aux_test = aux_test.drop('orig_destination_distance', axis=1)

In [ ]:
aux_test = aux_test.drop('date_time', axis=1)

In [ ]:
X_train.columns

In [ ]:
    
aux_test.columns

In [ ]:
test_pred = gbm.predict(aux_test)

print(test_pred)

In [ ]:
aux_test["predicted_ranking"] = test_pred


In [ ]:
aux_test.sort_values(["srch_id","predicted_ranking"], ascending=[True,False])

In [ ]:
aux_test2 = aux_test
aux_test2


In [ ]:
aux_test2.drop(aux_test2.columns.difference(['srch_id','prop_id']), 1, inplace=True)

In [ ]:
aux_test2.head(50)

In [ ]:
aux_test2.to_csv("out.csv", index=False)    

# -------------------------------------------------------------------------------------------------------------